In [1]:
!pip install fastai==2.7.1
!pip install timm>=0.6.2.dev0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 419.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 1.6 MB/s eta 0:00:00
  Attempting uninstall: fastcore
    Found existing installation: fastcore 1.5.6
    Uninstalling fastcore-1.5.6:
      Successfully uninstalled fastcore-1.5.6
  Attempting uninstall: fastai
    Found existing installation: fastai 2.7.7
    Uninstalling fastai-2.7.7:
      Successfully uninstalled fastai-2.7.7


In [2]:
from fastai.vision.all import *

In [3]:
train_path = '../input/paddy-disease-classification/train_images'
test_path = '../input/paddy-disease-classification/test_images'

In [4]:
test_files = get_image_files(test_path).sorted()

In [5]:
# Load training Data
dls = ImageDataLoaders.from_folder(train_path, valid_pct=0.2, item_tfms=Resize(640, 480),
        batch_tfms=aug_transforms(size=224, min_scale=0.75), bs=64//2)

In [6]:
learn = vision_learner(dls, 'vit_large_patch16_224', metrics=error_rate, cbs=GradientAccumulation(64)).to_fp16()

In [7]:
learn.fine_tune(5, 0.005)

epoch,train_loss,valid_loss,error_rate,time
0,1.048502,0.603751,0.199904,06:34


epoch,train_loss,valid_loss,error_rate,time
0,0.494400,0.329136,0.110043,08:53
1,0.372147,0.255575,0.068717,08:54
2,0.240472,0.162890,0.052859,08:53
3,0.133186,0.113397,0.036040,08:53
4,0.080501,0.111948,0.030274,08:54


In [8]:
preds,_ = learn.tta(dl=dls.test_dl(test_files))

In [9]:
idxs = preds.argmax(dim=1)

In [10]:
vocab = np.array(dls.vocab)
results = pd.Series(vocab[idxs], name="idxs")

In [11]:
submit = pd.read_csv('../input/paddy-disease-classification/sample_submission.csv')
submit['label'] = results
submit.to_csv('subm.csv', index=False)